In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import lime
import lime.lime_tabular

# 1. CSVファイルからデータを読み込む
# 日経平均株価データを 'japan_2022.csv' から読み込む
df = pd.read_csv('japan_2022.csv')

# データの先頭5行を表示して確認する
print("データの先頭5行:")
print(df.head())

データの先頭5行:
                  Date         Open         High          Low        Close  \
0  2022-01-04 00:00:00  29098.41016  29323.78906  28954.56055  29301.78906   
1  2022-01-05 00:00:00  29288.80078  29388.16016  29204.44922  29332.16016   
2  2022-01-06 00:00:00  29136.75000  29158.94922  28487.86914  28487.86914   
3  2022-01-07 00:00:00  28711.52930  28813.08984  28293.69922  28478.56055   
4  2022-01-11 00:00:00  28380.90039  28473.47070  28089.49023  28222.48047   

     Adj Close    Volume  
0  29301.78906  66000000  
1  29332.16016  78600000  
2  28487.86914  71600000  
3  28478.56055  75200000  
4  28222.48047  73700000  


In [3]:
# 2. 欠損値の確認
# 各列の欠損値の数を表示
print("\n欠損値の確認:")
print(df.isnull().sum())

# 3. 欠損値の補完
# 'ffill' は前の行の値で欠損値を埋める方法
df.fillna(method='ffill', inplace=True)

# 欠損値の補完後、再度欠損値の確認
print("\n欠損値補完後の確認:")
print(df.isnull().sum())

# 4. 移動平均の計算
# 5日, 10日, 20日の移動平均を計算して、特徴量として追加
df['MA_5'] = df['Close'].rolling(window=5).mean()
df['MA_10'] = df['Close'].rolling(window=10).mean()
df['MA_20'] = df['Close'].rolling(window=20).mean()

# 5. ボラティリティの計算
# 5日のボラティリティ（標準偏差）を計算して、特徴量として追加
df['Volatility'] = df['Close'].rolling(window=5).std()

# 6. 時間に関連する特徴量の追加
# 'Date'列を日時型に変換
df['Date'] = pd.to_datetime(df['Date'])
# 曜日 (0=月曜日, 6=日曜日)
df['DayOfWeek'] = df['Date'].dt.dayofweek
# 月 (1=1月, 12=12月)
df['Month'] = df['Date'].dt.month
# 週番号 (1から52または53)
df['WeekOfYear'] = df['Date'].dt.isocalendar().week

# 7. テクニカル指標の計算
# RSI (Relative Strength Index) の計算
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# 14日間のRSIを計算して追加
df['RSI'] = calculate_rsi(df)

# ボリンジャーバンドの計算
def calculate_bollinger_bands(data, window=20):
    sma = data['Close'].rolling(window=window).mean()
    rstd = data['Close'].rolling(window=window).std()
    upper_band = sma + 2 * rstd
    lower_band = sma - 2 * rstd
    return upper_band, lower_band

# 20日間のボリンジャーバンドを計算して追加
df['Upper_Band'], df['Lower_Band'] = calculate_bollinger_bands(df)

# MACD (Moving Average Convergence Divergence) の計算
# 12日EMA (Exponential Moving Average) と 26日EMA の差分を計算
df['MACD'] = df['Close'].ewm(span=12, adjust=False).mean() - df['Close'].ewm(span=26, adjust=False).mean()

# 9日EMA のシグナルラインを計算して追加
df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

# 8. ファンダメンタル指標の追加
# 出来高変化率 (Volume Change Rate) の計算
df['Volume_Change'] = df['Volume'].pct_change()
# 出来高の移動平均 (5日, 10日) の計算
df['Volume_MA_5'] = df['Volume'].rolling(window=5).mean()
df['Volume_MA_10'] = df['Volume'].rolling(window=10).mean()

# 9. 株価の変動率の計算
# 日次変動率 (Daily Price Change) の計算
df['Daily_Change'] = df['Close'].pct_change()
# 高値と安値の変動率 (High-Low Range) の計算
df['High_Low_Range'] = (df['High'] - df['Low']) / df['Low']

# 10. クラスラベルの作成
# 翌日の終値が今日の終値より高ければ1、低ければ0とする
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)

# 最後の行は目的変数（Target）が欠損しているため削除
df.dropna(inplace=True)

# 特徴量を含むデータの先頭5行を表示して確認
print("\n特徴量を追加したデータの先頭5行:")
print(df.head())


欠損値の確認:
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

欠損値補完後の確認:
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

特徴量を追加したデータの先頭5行:
         Date         Open         High          Low        Close  \
19 2022-02-01  27167.14063  27410.78906  27016.71094  27078.48047   
20 2022-02-02  27302.99023  27564.61914  27289.16016  27533.59961   
21 2022-02-03  27330.96094  27357.33008  27165.92969  27241.31055   
22 2022-02-04  27095.90039  27455.98047  27075.99023  27439.99023   
23 2022-02-07  27327.63086  27369.67969  27085.32031  27248.86914   

      Adj Close    Volume          MA_5         MA_10         MA_20  ...  \
19  27078.48047  81100000  26795.886328  27146.156055  27862.712989  ...   
20  27533.59961  85200000  26900.340234  27152.792969  27774.303517  ...   
21  27241.31055  81100000  27114.542188  27099.631055  27669.761036  ...   
22  2

In [4]:
# 11. モデル構築の準備
# 説明変数（特徴量）の選択
X = df[['Open', 'High', 'Low', 'Close', 'Volume', 'MA_5', 'MA_10', 'MA_20', 'Volatility',
        'DayOfWeek', 'Month', 'WeekOfYear', 'RSI', 'Upper_Band', 'Lower_Band', 'MACD', 
        'MACD_Signal', 'Volume_Change', 'Volume_MA_5', 'Volume_MA_10', 'Daily_Change', 
        'High_Low_Range']]

# 目的変数（ターゲット）の選択
y = df['Target']

# 訓練データとテストデータに分割（80%訓練データ、20%テストデータ）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ランダムフォレスト分類モデルの構築
model = RandomForestClassifier(n_estimators=100, random_state=42)

# モデルを訓練データで学習
model.fit(X_train, y_train)

# テストデータを用いて予測
y_pred = model.predict(X_test)

# 12. 評価
# 混同行列の作成
cm = confusion_matrix(y_test, y_pred)

# 正解率（Accuracy）の計算
accuracy = accuracy_score(y_test, y_pred)

# 精度（Precision）の計算
precision = precision_score(y_test, y_pred)

# 再現率（Recall）の計算
recall = recall_score(y_test, y_pred)

# F1スコアの計算
f1 = f1_score(y_test, y_pred)

# 評価結果を表示
print("\nモデルの評価:")
print(f'Confusion Matrix:\n{cm}')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')



モデルの評価:
Confusion Matrix:
[[ 6 14]
 [13 12]]
Accuracy: 0.4
Precision: 0.46153846153846156
Recall: 0.48
F1 Score: 0.47058823529411764


In [ ]:
# 特徴量の重要度を取得
feature_importances = model.feature_importances_

# 特徴量の重要度をデータフレームに変換
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# 重要度でソート
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 特徴量の重要度を棒グラフで表示
plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Feature Importances')
plt.show()


In [ ]:
# SHAP値の計算
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# SHAP Summary Plot
shap.summary_plot(shap_values[1], X_test, plot_type="bar")
shap.summary_plot(shap_values[1], X_test)

# 特定の予測のSHAP値を可視化
sample_index = 0  # 予測したいサンプルのインデックスを指定
shap.force_plot(explainer.expected_value[1], shap_values[1][sample_index], X_test.iloc[sample_index])


In [ ]:
# LIMEによる解釈
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X.columns,
    class_names=['Decrease', 'Increase'],
    mode='classification'
)

# 特定の予測のLIME解釈を表示
sample_index = 0  # 予測したいサンプルのインデックスを指定
lime_exp = lime_explainer.explain_instance(X_test.iloc[sample_index], model.predict_proba, num_features=10)

# LIME解釈の可視化
lime_exp.show_in_notebook()


In [ ]:
# 実際の値と予測値を比較するために、テストデータと予測値をプロット
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label='Actual', alpha=0.7)
plt.plot(y_pred, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted')
plt.legend()
plt.show()
